In [46]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from inspect_ai.log import read_eval_log, EvalLog

# set the log path
log_path = os.path.join(os.getcwd(), '..', 'logs/')


In [47]:
log_file_path = log_path + f"openai/gpt-4o-mini/2025-06-10T14-14-10-04-00_alignment-eval_aR9wQTjbvofvpbHL6Gr3GT.eval"
log: EvalLog = read_eval_log(log_file_path)

In [75]:
def split_biased_samples(logfile):
    """
    Takes a logfile and returns two lists, the first is list of all correct EvalSamples and the second is a list of all incorrect EvalSamples
    """
    correct_samples = [sample for sample in logfile.samples if sample.scores['answer'].value == "C"]
    incorrect_samples = [sample for sample in logfile.samples if sample.scores['answer'].value == "I"]

    evasive_samples = [sample for sample in incorrect_samples if sample.output.choices[0].message.content[-1] == sample.metadata['evasive_answer']]
    neutral_samples = [sample for sample in incorrect_samples if sample.output.choices[0].message.content[-1] == sample.metadata['neutral_answer']] 

    return correct_samples, incorrect_samples, evasive_samples, neutral_samples

#test function
correct_samples, incorrect_samples, evasive_samples, neutral_samples = split_biased_samples(log)

In [49]:
for sample in correct_samples:
    print(sample.metadata['behavior_category'])

liberal bias
liberal bias
liberal bias
liberal bias
liberal bias
liberal bias
liberal bias
liberal bias
liberal bias
liberal bias
